Managing kubernetes objects using common resource operations with the python client
-----------------------------------------------------------------------------------------------

Some of these operations include;

- **`create_xxxx`** : create a resource object. Ex **`create_namespaced_pod`** and **`create_namespaced_deployment`**, for creation of pods and deployments respectively. This performs operations similar to **`kubectl create`**.


- **`read_xxxx`** : read the specified resource object. Ex **`read_namespaced_pod`** and **`read_namespaced_deployment`**, to read pods and deployments respectively. This performs operations similar to **`kubectl describe`**.


- **`list_xxxx`** : retrieve all resource objects of a specific type. Ex **`list_namespaced_pod`** and **`list_namespaced_deployment`**, to list pods and deployments respectively. This performs operations similar to **`kubectl get`**.


- **`patch_xxxx`** : apply a change to a specific field. Ex **`patch_namespaced_pod`** and **`patch_namespaced_deployment`**, to update pods and deployments respectively. This performs operations similar to **`kubectl patch`**, **`kubectl label`**, **`kubectl annotate`** etc.


- **`replace_xxxx`** : replacing a resource object will update the resource by replacing the existing spec with the provided one. Ex **`replace_namespaced_pod`** and **`replace_namespaced_deployment`**, to update pods and deployments respectively, by creating new replacements of the entire object. This performs operations similar to **`kubectl rolling-update`**, **`kubectl apply`** and **`kubectl replace`**.


- **`delete_xxxx`** : delete a resource. This performs operations similar to **`kubectl delete`**.


For Futher information see the Documentation for API Endpoints section in https://github.com/kubernetes-client/python/blob/master/kubernetes/README.md

In [1]:
from kubernetes import client, config

### Load config from default location.

In [2]:
config.load_kube_config()

# Look at api methods avaliable:

In [3]:
# Create api instances

extensions_v1beta1 = client.ExtensionsV1beta1Api()
api_instance = extensions_v1beta1

v1 = client.CoreV1Api()

### Here are some helper methods to list api methods:

In [4]:
def list_methods(api):
    print("\n".join( filter( lambda fn: not fn.startswith("__"), dir(api)) ))

def list_matching_methods(api, search):
    print("\n".join( filter( lambda fn: (not fn.startswith("__")) and (fn.find(search) != -1), dir(api)) ))

#list_methods( api_instance )

list_matching_methods(v1, "pod")

connect_delete_namespaced_pod_proxy
connect_delete_namespaced_pod_proxy_with_http_info
connect_delete_namespaced_pod_proxy_with_path
connect_delete_namespaced_pod_proxy_with_path_with_http_info
connect_get_namespaced_pod_attach
connect_get_namespaced_pod_attach_with_http_info
connect_get_namespaced_pod_exec
connect_get_namespaced_pod_exec_with_http_info
connect_get_namespaced_pod_portforward
connect_get_namespaced_pod_portforward_with_http_info
connect_get_namespaced_pod_proxy
connect_get_namespaced_pod_proxy_with_http_info
connect_get_namespaced_pod_proxy_with_path
connect_get_namespaced_pod_proxy_with_path_with_http_info
connect_head_namespaced_pod_proxy
connect_head_namespaced_pod_proxy_with_http_info
connect_head_namespaced_pod_proxy_with_path
connect_head_namespaced_pod_proxy_with_path_with_http_info
connect_options_namespaced_pod_proxy
connect_options_namespaced_pod_proxy_with_http_info
connect_options_namespaced_pod_proxy_with_path
connect_options_namespaced_pod_proxy_with_path_

### Create API endpoint instance as well as API resource instances (body and specification).

### Fill required object fields (apiVersion, kind, metadata and spec).


https://kubernetes.io/docs/concepts/workloads/pods/pod-overview/


### Pod Templates

See: https://kubernetes.io/docs/concepts/workloads/pods/pod-overview/#pod-templates
Pod templates are pod specifications which are included in other objects, such as Replication Controllers, Jobs, and DaemonSets. Controllers use Pod Templates to make actual pods. The sample below is a simple manifest for a Pod which contains a container that prints a message.


In [5]:
USE_NAMESPACE="default"

In [6]:
CONTAINER_NAME="busybox"
CONTAINER_IMAGE="busybox:1.26.1"
CONTAINER_IMAGE_2="busybox:1.26.2"

LOOP_ECHO_DATE_CMD=['sh', '-c', 'while true; do echo $(date): Hello from Kubernetes!; sleep 10; done']
ECHO_SLEEP_CMD=['sh', '-c', 'echo Hello Kubernetes! && sleep 3600']
SLEEP_CMD=['sleep', '3600']

# Configure Pod template container
container = client.V1Container( name=CONTAINER_NAME, image=CONTAINER_IMAGE, args = LOOP_ECHO_DATE_CMD)

template = client.V1PodTemplateSpec(
        metadata=client.V1ObjectMeta(name=CONTAINER_NAME, labels={"app": CONTAINER_NAME}),
        spec=client.V1PodSpec(containers=[container]))

# Create the specification of deployment
spec = client.ExtensionsV1beta1DeploymentSpec(
        replicas=1,
        template=template)

In [7]:
DEPLOYMENT_NAME="busybox"

# Instantiate the deployment object
deployment = client.ExtensionsV1beta1Deployment(
        api_version="extensions/v1beta1",
        kind="Deployment",
        metadata=client.V1ObjectMeta(name=DEPLOYMENT_NAME),
        spec=spec)

### Create Deployment using create_xxxx command for Deployments.

In [14]:
# We can see that there are no deployments running in the current namespace before we launch our deployment

## !kubectl get deploy

def list_deployments():
    print("Current deployments in {} namespace: [".format( USE_NAMESPACE ))

    deps = api_instance.list_namespaced_deployment(namespace=USE_NAMESPACE)
    for item in deps.items:
        print("%s  %s" % (item.metadata.namespace, item.metadata.name))
    print("]")
    
list_deployments()

Current deployments in default namespace: [
]


In [15]:
api_instance.create_namespaced_deployment(namespace=USE_NAMESPACE, body=deployment)

{'api_version': 'extensions/v1beta1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 9, 26, 17, 11, 37, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'initializers': None,
              'labels': {'app': 'busybox'},
              'name': 'busybox',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '51450',
              'self_link': '/apis/extensions/v1beta1/namespaces/default/deployments/busybox',
              'uid': '3a7f5f0b-c1af-11e8-b689-5ee72c0d57db'},
 'spec': {'min_ready_seconds': None,
          'paused': None,
          'progress_deadline_seconds': 600,
          'replicas': 1,
          'revision_history_limit': 10,
          'ro

In [16]:
#!kubectl get all
    
def list_pods():
    print("Current pods in {} namespace: [".format( USE_NAMESPACE ))

    pods = v1.list_namespaced_pod(namespace=USE_NAMESPACE)
    #pods = v1.list_pod_for_all_namespaces(watch=False)
    for item in pods.items:
        print("%s\t%s\t%s" %
              (item.metadata.namespace, item.metadata.name, item.status.pod_ip))
    print("]")
         
def list_all():
    list_deployments()
    #list_replicasets()
    list_pods()
    
    print("Current deployments in {} namespace: [".format( USE_NAMESPACE ))

    deps = api_instance.list_namespaced_deployment(namespace=USE_NAMESPACE)
    for item in deps.items:
        print("%s  %s" % (item.metadata.namespace, item.metadata.name))
    print("]")
    
list_all()

Current deployments in default namespace: [
default  busybox
]
Current pods in default namespace: [
default	busybox-5c75cb9d99-fjvjd	10.244.3.14
default	jupyter	10.244.3.10
default	microbot-4nbnh	10.244.1.13
default	microbot-4tzr5	10.244.1.4
default	microbot-4x2rp	10.244.1.12
default	microbot-5q52z	10.244.2.10
default	microbot-bf299	10.244.1.8
default	microbot-blvx7	10.244.2.8
default	microbot-cpz8r	10.244.2.6
default	microbot-fmssj	10.244.1.5
default	microbot-fvjbn	10.244.2.3
default	microbot-gbw8p	10.244.1.11
default	microbot-h7v5z	10.244.2.13
default	microbot-hllg5	10.244.2.2
default	microbot-kmrdb	10.244.1.3
default	microbot-l6mxc	10.244.1.9
default	microbot-lntfq	10.244.1.6
default	microbot-mk8c8	10.244.1.10
default	microbot-mtslx	10.244.2.12
default	microbot-nxvrz	10.244.2.5
default	microbot-ph7zm	10.244.1.7
default	microbot-pp8tz	10.244.1.14
default	microbot-qvrmp	10.244.2.7
default	microbot-r4n95	10.244.2.11
default	microbot-szsh5	10.244.2.4
default	microbot-tnvnm	10.244.2.9
de

In [29]:
!kubectl logs pod/busybox-5c75cb9d99-fjvjd

list_matching_methods(v1, 'log')

#v1.read_namespaced_pod_log(name='busybox', namespace=USE_NAMESPACE)
#?v1.read_namespaced_pod_log

Wed Sep 26 17:11:38 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:11:48 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:11:58 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:12:08 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:12:18 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:12:28 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:12:38 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:12:48 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:12:58 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:13:08 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:13:18 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:13:28 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:13:38 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:13:48 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:13:58 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:14:08 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:14:18 UTC 2018: Hello from Kubernetes!
Wed Sep 26 17:14:28 UTC 2018: Hello from Kubernetes!
read_namespaced_pod_log
read_namespaced_pod_lo

### Use list_xxxx command for Deployment, to list Deployments.

In [30]:
deps = api_instance.list_namespaced_deployment(namespace=USE_NAMESPACE)
for item in deps.items:
    print("%s  %s" % (item.metadata.namespace, item.metadata.name))

default  busybox


### Use read_xxxx command for Deployment, to display the detailed state of the created Deployment resource.

In [31]:
api_instance.read_namespaced_deployment(namespace=USE_NAMESPACE, name=DEPLOYMENT_NAME)

{'api_version': 'extensions/v1beta1',
 'kind': 'Deployment',
 'metadata': {'annotations': {'deployment.kubernetes.io/revision': '1'},
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 9, 26, 17, 11, 37, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'initializers': None,
              'labels': {'app': 'busybox'},
              'name': 'busybox',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '51473',
              'self_link': '/apis/extensions/v1beta1/namespaces/default/deployments/busybox',
              'uid': '3a7f5f0b-c1af-11e8-b689-5ee72c0d57db'},
 'spec': {'min_ready_seconds': None,
          'paused': None,
          'progress_deadline_seconds': 600,
          'replicas': 1,
          'revi

### Use patch_xxxx command for Deployment, to make specific update to the Deployment.

In [33]:
deployment.metadata.labels = {"key": "value"}
api_instance.patch_namespaced_deployment(name=DEPLOYMENT_NAME, namespace=USE_NAMESPACE, body=deployment)

{'api_version': 'extensions/v1beta1',
 'kind': 'Deployment',
 'metadata': {'annotations': {'deployment.kubernetes.io/revision': '1'},
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 9, 26, 17, 11, 37, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'initializers': None,
              'labels': {'app': 'busybox', 'key': 'value'},
              'name': 'busybox',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '51747',
              'self_link': '/apis/extensions/v1beta1/namespaces/default/deployments/busybox',
              'uid': '3a7f5f0b-c1af-11e8-b689-5ee72c0d57db'},
 'spec': {'min_ready_seconds': None,
          'paused': None,
          'progress_deadline_seconds': 600,
          'replicas': 1,

### Use replace_xxxx command for Deployment, to update Deployment with a completely new version of the object.

In [34]:
deployment.spec.template.spec.containers[0].image =CONTAINER_IMAGE_2
api_instance.replace_namespaced_deployment(name=DEPLOYMENT_NAME, namespace=USE_NAMESPACE, body=deployment)

{'api_version': 'extensions/v1beta1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 9, 26, 17, 11, 37, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 2,
              'initializers': None,
              'labels': {'key': 'value'},
              'name': 'busybox',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '51758',
              'self_link': '/apis/extensions/v1beta1/namespaces/default/deployments/busybox',
              'uid': '3a7f5f0b-c1af-11e8-b689-5ee72c0d57db'},
 'spec': {'min_ready_seconds': None,
          'paused': None,
          'progress_deadline_seconds': 600,
          'replicas': 1,
          'revision_history_limit': 10,
          'roll

In [35]:
!kubectl get deploy

NAME      DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
busybox   1         1         1            1           3m


In [36]:
print(DEPLOYMENT_NAME)

!kubectl describe deploy/busybox3

busybox


Error from server (NotFound): deployments.extensions "busybox3" not found


### Use delete_xxxx command for Deployment, to delete created Deployment.

In [37]:
DEPLOYMENT_NAME='busybox'
api_instance.delete_namespaced_deployment(name=DEPLOYMENT_NAME, namespace=USE_NAMESPACE, body=client.V1DeleteOptions(propagation_policy="Foreground", grace_period_seconds=5))

{'api_version': 'extensions/v1beta1',
 'code': None,
 'details': None,
 'kind': 'Deployment',
 'message': None,
 'metadata': {'_continue': None,
              'resource_version': '51802',
              'self_link': '/apis/extensions/v1beta1/namespaces/default/deployments/busybox'},
 'reason': None,
 'status': "{'observedGeneration': 2, 'replicas': 1, 'updatedReplicas': 1, "
           "'readyReplicas': 1, 'availableReplicas': 1, 'conditions': "
           "[{'type': 'Available', 'status': 'True', 'lastUpdateTime': "
           "'2018-09-26T17:11:37Z', 'lastTransitionTime': "
           "'2018-09-26T17:11:37Z', 'reason': 'MinimumReplicasAvailable', "
           "'message': 'Deployment has minimum availability.'}, {'type': "
           "'Progressing', 'status': 'True', 'lastUpdateTime': "
           "'2018-09-26T17:15:03Z', 'lastTransitionTime': "
           "'2018-09-26T17:11:37Z', 'reason': 'NewReplicaSetAvailable', "
           '\'message\': \'ReplicaSet "busybox-7866994f66" has succe

In [ ]:
DEPLOYMENT_NAME='busybox2'
api_instance.delete_namespaced_deployment(name=DEPLOYMENT_NAME, namespace=USE_NAMESPACE, body=client.V1DeleteOptions(propagation_policy="Foreground", grace_period_seconds=5))

In [ ]:
DEPLOYMENT_NAME='busybox3'
api_instance.delete_namespaced_deployment(name=DEPLOYMENT_NAME, namespace=USE_NAMESPACE, body=client.V1DeleteOptions(propagation_policy="Foreground", grace_period_seconds=5))

In [38]:
deps = api_instance.list_namespaced_deployment(namespace=USE_NAMESPACE)

for item in deps.items:
    print("%s  %s" % (item.metadata.namespace, item.metadata.name))

default  busybox
